# Preparation of Dataset for solubility prediction

## Datasets

1. [ESOL: Estimating Aqueous Solubility Directly from Molecular Structure](https://pubs.acs.org/doi/10.1021/ci034243x)
2. [AqSolDB, a curated reference set of aqueous solubility and 2D descriptors for a diverse set of compounds](https://www.nature.com/articles/s41597-019-0151-1)
3. [Is Experimental Data Quality the Limiting Factor in Predicting the Aqueous Solubility of Driglike Molecules](https://pubs.acs.org/doi/full/10.1021/mp500103r)

## Backgroud:

Aqueous solubility is a key factor in drug discovery, since if a molecule is not soluble. it will typically be poorly bioavailable, making it difficult to perform <i>in-vivo</i> studies with it, and hence deliver to patients.

In [145]:
! pip install rdkit pandas numpy

In [211]:
import math
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

In [147]:
def load_csv(path: str) -> pd.DataFrame:
    return pd.read_csv(path)

In [148]:
esol = load_csv(path="data/esol.csv")
aqsol = load_csv(path="data/solubility-dataset-train.csv")
dsl = load_csv(path="data/dsl-100-unique.csv")

In [149]:
print(esol.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 10 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   Compound ID                                      1128 non-null   object 
 1   ESOL predicted log solubility in mols per litre  1128 non-null   float64
 2   Minimum Degree                                   1128 non-null   int64  
 3   Molecular Weight                                 1128 non-null   float64
 4   Number of H-Bond Donors                          1128 non-null   int64  
 5   Number of Rings                                  1128 non-null   int64  
 6   Number of Rotatable Bonds                        1128 non-null   int64  
 7   Polar Surface Area                               1128 non-null   float64
 8   measured log solubility in mols per litre        1128 non-null   float64
 9   smiles                        

In [150]:
print(aqsol.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9982 entries, 0 to 9981
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   9982 non-null   object 
 1   Name                 9982 non-null   object 
 2   InChI                9982 non-null   object 
 3   InChIKey             9982 non-null   object 
 4   SMILES               9982 non-null   object 
 5   Solubility           9982 non-null   float64
 6   SD                   9982 non-null   float64
 7   Ocurrences           9982 non-null   int64  
 8   Group                9982 non-null   object 
 9   MolWt                9982 non-null   float64
 10  MolLogP              9982 non-null   float64
 11  MolMR                9982 non-null   float64
 12  HeavyAtomCount       9982 non-null   float64
 13  NumHAcceptors        9982 non-null   float64
 14  NumHDonors           9982 non-null   float64
 15  NumHeteroatoms       9982 non-null   f

In [151]:
print(dsl.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     56 non-null     int64  
 1   Chemical name                  56 non-null     object 
 2   Crystal structure CSD refcode  56 non-null     object 
 3   LogS exp (mol/L)               56 non-null     float64
 4   Reference                      56 non-null     int64  
 5   Test                           56 non-null     bool   
 6   SMILES                         56 non-null     object 
 7   Smiles Source                  56 non-null     object 
 8   Chemspider reference number    56 non-null     int64  
 9   InChl                          56 non-null     object 
 10  cansmi                         56 non-null     object 
 11  is_unique                      56 non-null     bool   
dtypes: bool(2), float64(1), int64(3), object(6)
memory u

## Calculating Molecular Descriptors

To predict log(S), the study of Delaney used four molecular descriptors:
- cLogP (Octanol-Water partition coefficient)
- MW (Molecular Weight)
- RB (Number of rotatable bonds)
- AP (Aromatic proportion = Number of aromatic atoms / total number of heavy atoms)

In [152]:
def get_number_aromatic_atoms(molecule) -> int:
    
    aromatic_atoms = [molecule.GetAtomWithIdx(idx).GetIsAromatic() for idx in range(molecule.GetNumAtoms())]
    count = []
    for _, value in enumerate(aromatic_atoms):
        if value == True:
            count.append(value)
    sum_count = sum(count)
    
    return sum_count

In [153]:
def calc_aromatic_descriptors(molecule) -> float:
    aromatic_atoms: int = get_number_aromatic_atoms(molecule)
    num_heavy_atoms: int = Descriptors.HeavyAtomCount(molecule)
    
    aromatic_proportion: float = aromatic_atoms / num_heavy_atoms
    return aromatic_proportion

In [154]:
def generate_dataset(molecules) -> pd.DataFrame:
    smiles = []
    rows = []

    for molecule in molecules:
        descriptor = Chem.MolFromSmiles(molecule)
        smiles.append(descriptor)
    
    for smile in smiles:
        mol_log_p: float = Descriptors.MolLogP(smile)
        mol_weight: float = Descriptors.MolWt(smile)
        num_rot_bonds: int = Descriptors.NumRotatableBonds(smile)
        aromatic_proportion: float = calc_aromatic_descriptors(smile)
        
        row = [smile, mol_log_p, mol_weight, num_rot_bonds, aromatic_proportion]
        rows.append(row)
    
    df = pd.DataFrame(data=rows, columns=["SMILES", "MolLogP", "MolWeight", "NumRotatableBonds", "AromaticProportion"])
    return df

In [155]:
esol_parsed = generate_dataset(molecules=esol["smiles"])
aqsol_parsed = generate_dataset(molecules=aqsol["SMILES"])
dsl_parsed = generate_dataset(molecules=dsl["SMILES"])

[16:48:01] WARNING: not removing hydrogen atom without neighbors
[16:48:01] WARNING: not removing hydrogen atom without neighbors
[16:48:01] WARNING: not removing hydrogen atom without neighbors
[16:48:01] WARNING: not removing hydrogen atom without neighbors
[16:48:01] WARNING: not removing hydrogen atom without neighbors
[16:48:01] WARNING: not removing hydrogen atom without neighbors
[16:48:01] WARNING: not removing hydrogen atom without neighbors
[16:48:01] WARNING: not removing hydrogen atom without neighbors
[16:48:01] WARNING: not removing hydrogen atom without neighbors
[16:48:01] WARNING: not removing hydrogen atom without neighbors
[16:48:01] WARNING: not removing hydrogen atom without neighbors
[16:48:01] WARNING: not removing hydrogen atom without neighbors
[16:48:02] WARNING: not removing hydrogen atom without neighbors
[16:48:02] WARNING: not removing hydrogen atom without neighbors
[16:48:02] WARNING: not removing hydrogen atom without neighbors
[16:48:02] WARNING: not r

In [156]:
esol.columns.values # y = measured log solubility in mols / L

array(['Compound ID', 'ESOL predicted log solubility in mols per litre',
       'Minimum Degree', 'Molecular Weight', 'Number of H-Bond Donors',
       'Number of Rings', 'Number of Rotatable Bonds',
       'Polar Surface Area', 'measured log solubility in mols per litre',
       'smiles'], dtype=object)

In [157]:
aqsol.columns.values # y = Solubility (reported as LogS (mol/L))

array(['ID', 'Name', 'InChI', 'InChIKey', 'SMILES', 'Solubility', 'SD',
       'Ocurrences', 'Group', 'MolWt', 'MolLogP', 'MolMR',
       'HeavyAtomCount', 'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms',
       'NumRotatableBonds', 'NumValenceElectrons', 'NumAromaticRings',
       'NumSaturatedRings', 'NumAliphaticRings', 'RingCount', 'TPSA',
       'LabuteASA', 'BalabanJ', 'BertzCT'], dtype=object)

In [158]:
dsl.columns.values # y = LogS exp (mol/L)

array(['Unnamed: 0', 'Chemical name', 'Crystal structure CSD refcode',
       'LogS exp (mol/L)', 'Reference', 'Test', 'SMILES', 'Smiles Source',
       'Chemspider reference number', 'InChl', 'cansmi', 'is_unique'],
      dtype=object)

In [159]:
esol_parsed["y"] = esol["measured log solubility in mols per litre"]
aqsol_parsed["y"] = aqsol["Solubility"]
dsl_parsed["y"] = dsl["LogS exp (mol/L)"]

In [160]:
train_dataset = pd.concat([esol_parsed, aqsol_parsed], axis=0)
train_dataset.head()

train_dataset = train_dataset.sample(frac=1)

In [161]:
print("Training dataset: ", train_dataset.shape)
print("Testing dataset: ", dsl_parsed.shape)

Training dataset:  (11110, 6)
Testing dataset:  (56, 6)


In [162]:
train_dataset[train_dataset.duplicated()]

,SMILES,MolLogP,MolWeight,NumRotatableBonds,AromaticProportion,y


In [171]:
! pip install scikit-learn lightgbm xgboost

In [212]:
from xgboost.sklearn import XGBRegressor
from lightgbm.sklearn import LGBMRegressor


from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

X_dataset, y = train_dataset[["MolLogP", "MolWeight", "NumRotatableBonds", "AromaticProportion"]], train_dataset["y"]


models = {
    "linear-regression": LinearRegression().fit(X_dataset, y),
    "random-forest": RandomForestRegressor().fit(X_dataset, y),
    "lightgbm-regressor": LGBMRegressor().fit(X_dataset, y),
    "xgboost-regressor": XGBRegressor().fit(X_dataset, y)
}

for key, model in models.items():
    print(f"Training dataset statistics: {key} \n")
    
    pred_train = model.predict(X_dataset)
    
    if key == "linear-regression":
        print("Coefficients: ", model.coef_)
        print("Intercept: ", model.intercept_)
        
    print(f"RMSE: {math.sqrt(mean_squared_error(y, pred_train)):.3f}")
    print(f"MAE: {mean_absolute_error(y, pred_train):.3f}")
    print(f"Coefficient of Correlation: {r2_score(y, pred_train):.3f}")
    
    print("===============================================\n")



Training dataset statistics: linear-regression 

Coefficients:  [-0.40006646 -0.00413119  0.07682077 -0.74904093]
Intercept:  -1.078668523504993
RMSE: 1.716
MAE: 1.254
Coefficient of Correlation: 0.464

Training dataset statistics: random-forest 

RMSE: 0.476
MAE: 0.318
Coefficient of Correlation: 0.959

Training dataset statistics: lightgbm-regressor 

RMSE: 1.098
MAE: 0.777
Coefficient of Correlation: 0.780

Training dataset statistics: xgboost-regressor 

RMSE: 0.877
MAE: 0.627
Coefficient of Correlation: 0.860



In [213]:
X_test, y_test = dsl_parsed[["MolLogP", "MolWeight", "NumRotatableBonds", "AromaticProportion"]], dsl_parsed["y"]

for key, model in models.items():
    print(f"Training dataset statistics: {key} \n")
    
    pred_train = model.predict(X_test)
        
    print(f"RMSE: {math.sqrt(mean_squared_error(y_test, pred_train)):.3f}")
    print(f"MAE: {mean_absolute_error(y_test, pred_train):.3f}")
    print(f"Coefficient of Correlation: {r2_score(y_test, pred_train):.3f}")
    
    print("===============================================\n")

Training dataset statistics: linear-regression 

RMSE: 1.122
MAE: 0.883
Coefficient of Correlation: 0.543

Training dataset statistics: random-forest 

RMSE: 0.658
MAE: 0.430
Coefficient of Correlation: 0.843

Training dataset statistics: lightgbm-regressor 

RMSE: 0.903
MAE: 0.711
Coefficient of Correlation: 0.704

Training dataset statistics: xgboost-regressor 

RMSE: 0.938
MAE: 0.712
Coefficient of Correlation: 0.680



In [214]:
print("Baseline model: Linear regression \n")
pred_train = models["linear-regression"].predict(X_test)
        
print(f"RMSE: {math.sqrt(mean_squared_error(y_test, pred_train)):.3f}")
print(f"MAE: {mean_absolute_error(y_test, pred_train):.3f}")
print(f"Coefficient of Correlation: {r2_score(y_test, pred_train):.3f}")

Baseline model: Linear regression 

RMSE: 1.122
MAE: 0.883
Coefficient of Correlation: 0.543


In [215]:
print("Best model: Random Forest Regression \n")
pred_train = models["random-forest"].predict(X_test)
        
print(f"RMSE: {math.sqrt(mean_squared_error(y_test, pred_train)):.3f}")
print(f"MAE: {mean_absolute_error(y_test, pred_train):.3f}")
print(f"Coefficient of Correlation: {r2_score(y_test, pred_train):.3f}")

Best model: Random Forest Regression 

RMSE: 0.658
MAE: 0.430
Coefficient of Correlation: 0.843


In [ ]:
train_dataset.to_csv("data/esol-aqsol-training-set.csv", index=False)
dsl.to_csv("data/dsl-100-test-set.csv", index=False)

### Use optuna for hyperparameter tuning

In [181]:
! pip install optuna

     -------------------------------------- 348.5/348.5 kB 4.3 MB/s eta 0:00:00
     ---------------------------------------- 210.4/210.4 kB ? eta 0:00:00
     ---------------------------------------- 81.0/81.0 kB 4.4 MB/s eta 0:00:00
  Using cached colorlog-6.7.0-py2.py3-none-any.whl (11 kB)
     ---------------------------------------- 1.6/1.6 MB 19.9 MB/s eta 0:00:00
     ---------------------------------------- 78.7/78.7 kB 4.3 MB/s eta 0:00:00
     ------------------------------------- 190.9/190.9 kB 12.0 MB/s eta 0:00:00
  Using cached prettytable-3.5.0-py3-none-any.whl (26 kB)
     ---------------------------------------- 50.0/50.0 kB ? eta 0:00:00
  Using cached autopage-0.5.1-py3-none-any.whl (29 kB)
  Using cached cmd2-2.4.2-py3-none-any.whl (147 kB)
  Using cached pyperclip-1.8.2.tar.gz (20 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pyreadline3-3.4.1-py3-none-any.whl (95 kB)
     ------------------

In [195]:
import datetime
import optuna
from functools import partial

from sklearn.model_selection import cross_val_score

def objective(trial, name, X_train, y_train):
    max_depth = trial.suggest_int('max_depth', 1, 100)
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    criterion = trial.suggest_categorical("criterion", ["squared_error", "absolute_error", "friedman_mse"])
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    
    random_regressor = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        criterion=criterion,
        max_features=max_features
    )
    
    score = cross_val_score(random_regressor, X_train, y_train, cv=10)
    return score.mean()

current_time = datetime.datetime.now()

opt_function = partial(objective, name="rf", X_train=X_dataset, y_train=y)

study = optuna.create_study(direction="maximize", study_name=f"experiment-{current_time}")
study.optimize(opt_function, n_trials=30, n_jobs=-1)                                                         

[I 2022-12-30 22:12:12,872] A new study created in memory with name: experiment-2022-12-30 22:12:12.872330
[I 2022-12-30 22:12:15,542] Trial 2 finished with value: 0.6625445286688569 and parameters: {'max_depth': 4, 'n_estimators': 28, 'criterion': 'squared_error', 'max_features': 'log2'}. Best is trial 2 with value: 0.6625445286688569.
[I 2022-12-30 22:12:56,128] Trial 3 finished with value: 0.7287807455300485 and parameters: {'max_depth': 63, 'n_estimators': 71, 'criterion': 'friedman_mse', 'max_features': None}. Best is trial 3 with value: 0.7287807455300485.
[I 2022-12-30 22:13:43,311] Trial 5 finished with value: 0.7301453191440814 and parameters: {'max_depth': 94, 'n_estimators': 83, 'criterion': 'friedman_mse', 'max_features': None}. Best is trial 5 with value: 0.7301453191440814.
[I 2022-12-30 22:14:17,002] Trial 6 finished with value: 0.72887479962535 and parameters: {'max_depth': 33, 'n_estimators': 59, 'criterion': 'friedman_mse', 'max_features': None}. Best is trial 5 with 

In [207]:
params = study.best_params

random_forest = RandomForestRegressor(**params).fit(X_dataset, y)

pred_train = random_forest.predict(X_dataset)
        
print(f"RMSE: {math.sqrt(mean_squared_error(y, pred_train)):.3f}")
print(f"MAE: {mean_absolute_error(y, pred_train):.3f}")
print(f"Coefficient of Correlation: {r2_score(y, pred_train):.3f}")

RMSE: 0.491
MAE: 0.329
Coefficient of Correlation: 0.956


In [208]:
pred_test = random_forest.predict(X_test)
        
print(f"RMSE: {math.sqrt(mean_squared_error(y_test, pred_test)):.3f}")
print(f"MAE: {mean_absolute_error(y_test, pred_test):.3f}")
print(f"Coefficient of Correlation: {r2_score(y_test, pred_test):.3f}")

RMSE: 0.614
MAE: 0.408
Coefficient of Correlation: 0.863


In [204]:
import joblib

with open(file="model/regressors/solubility-rf-sklearn.model", mode="wb") as file:
    joblib.dump(value=random_forest, filename=file)

In [205]:
import torch